<a href="https://colab.research.google.com/github/mavela/text-classification-with-svm/blob/main/text_classification_long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Let's get data from Github first

## Text classification on web registers -- longer version
Steps

1.   Get data from Github
2.   Have a look at conllu data
3. Extract specific registers (you can choose!)
4. Featurize 
5. Divide to train and test
6. Run the SVM

## Things to analyze
1. Do the results vary between different registers?
2. Do the results vary between feature sets?






In [ ]:
! git clone https://github.com/mavela/text-classification-with-svm.git


In [ ]:
%cd text-classification-with-svm/
! ls

## Let's see how it looks like - print 20 first lines

In [ ]:
! zcat pb_smallpart.conllu.gz | head -20

### Fetch register data from the parsebank file -- decide which ones you want to try!

First we must extract our text classes from the parsebank file. The following classes are available:

* how-to_instructions
* informational_description
* informational_persuasion_general
* interactive_discussion
* machine_translated
* narrative
* opinion

In [ ]:
from analyze import extract_register

extract_register("narrative", "pb_smallpart.conllu.gz")

extract_register("interactive_discussion", "pb_smallpart.conllu.gz")
! ls

In [ ]:
!head -10 interactive_discussion_ext.conllu # if you want to have a look at the data, run this!

## Featurization
We can extract the features we want by retrieving information on a specific column in the conllu format. 

The columns are identified as ID, FORM, LEMMA, UPOS, XPOS, FEAT, HEAD, DEPREL, DEPS, MISC

In [11]:
from svm_scripts import save_text_format

save_text_format("narrative_ext.conllu","LEMMA") 
save_text_format("interactive_discussion_ext.conllu", "LEMMA")




In [ ]:
! head -5 interactive_discussion_ext.conllu_out.txt

In [ ]:
! wc -l narrative_ext.conllu_out.txt # here we print how many lines the featurized files have. oups, a bit of an imbalance! 
! wc -l interactive_discussion_ext.conllu_out.txt

### Then let's divide the data to train and text, and give register labels to the texts

In [ ]:
! cat narrative_ext.conllu_out.txt | head -500 | perl -pe 's/^/NA\t/g' > na_train.txt
! cat interactive_discussion_ext.conllu_out.txt | head -500 | perl -pe 's/^/ID\t/g' >  id_train.txt

! cat narrative_ext.conllu_out.txt | tail -200 | perl -pe 's/^/NA\t/g' > na_test.txt
! cat interactive_discussion_ext.conllu_out.txt | tail -200 | perl -pe 's/^/ID\t/g' > id_test.txt

! cat id_test.txt | head -3

In [ ]:
! cat na_train.txt id_train.txt | shuf  > train.txt
! cat na_test.txt id_test.txt | shuf > test.txt
! wc -l train.txt
! wc -l test.txt

## Then finally the classification!

You can see if / how the performance varies if you replace the words with syntactic information


In [ ]:
! python3 svm.py test.txt train.txt

## Want to understand how and why the classifier makes its decisions?

From a simple linear classifier such as the SVM we are using here, we can print the most important features identified by the classifier.

In [ ]:
! python3 svm_explain.py test.txt train.txt